In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool, Tool, BaseTool
from pydantic import BaseModel, Field
from typing import Any, Type
from langchain.utilities import DuckDuckGoSearchAPIWrapper ##검색라이브러리
import os
import requests
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo-0125"
)

alpha_vantage_api_key=os.environ.get("ALPHA_VANTAGE_API_KEY")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )

class StockMarketSymbolSearchTool(BaseTool):
    name="StockMarketSymbolSearchTool"
    description=""" 
        Use this tool to find the stock market symbol for a company.
        It takes a query as an argument.
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema
    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)




class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )

class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]

class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()

class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_ANNUALY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()

agent = initialize_agent(llm=llm, 
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_error=True, ## 자동으로 오류 해결
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool()
    ]
    )


prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."


response = agent.invoke(prompt)
response



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare Company'}`


Find the latest Cloudflare, Inc. (NET) stock quote, history, news and other vital information to help you with your stock trading and investing. Get the latest Cloudflare Inc (NET) real-time quote, historical performance, charts, and other financial information to help you make more informed trading and investment decisions. What Is the Cloudflare Inc Stock Price Today? The Cloudflare Inc stock price today is 111.58. What Is the Stock Symbol for Cloudflare Inc? The stock ticker symbol for Cloudflare... Cloudflare's market capitalization is $33.26 B by 305.71 M shares outstanding. Is Cloudflare stock a Buy, Sell or Hold? Cloudflare stock has received a consensus rating of hold. NET | Complete Cloudflare Inc. Cl A stock news by MarketWatch. View real-time stock prices and stock quotes for a full financial overview.
Invoking: `CompanyOverview` w

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "### Cloudflare Inc. (NET) Financial Analysis:\n\n#### Company Overview:\n- **Symbol:** NET\n- **Description:** Cloudflare, Inc. operates a cloud platform that offers a range of network services to companies worldwide.\n- **Market Capitalization:** $38.64 billion\n- **Sector:** Technology\n- **Industry:** Services-Prepackaged Software\n- **Revenue (TTM):** $1.57 billion\n- **Gross Profit (TTM):** $742.63 million\n- **EPS (TTM):** -$0.27\n- **Profit Margin:** -5.97%\n- **Operating Margin (TTM):** -7.16%\n- **Return on Assets (TTM):** -3.27%\n- **Return on Equity (TTM):** -11.2%\n- **Analyst Target Price:** $97.82\n- **52-Week High:** $119.42\n- **52-Week Low:** $66.24\n\n#### Income Statement:\n- **Latest Fiscal Year End:** December 31, 2023\n- **Revenue (2023):** $1.30 billion\n- **Gross Pro